### Key-Value RDD로 평균 구하기
레스토랑 데이터에서 category별 reviews 평균구하기

In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local').setAppName('review-average')
sc = SparkContext(conf = conf)

24/04/02 15:05:21 WARN Utils: Your hostname, MZC01-HYUCKSANGCHO.local resolves to a loopback address: 127.0.0.1; using 10.90.8.26 instead (on interface en0)
24/04/02 15:05:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/02 15:05:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
lines = sc.textFile("file:///Users/mzc01-hyucksangcho/Desktop/python/Spark/강의/빅데이터 처리를 위한 Spark/data/restaurant_reviews.csv")
lines.collect()

['id,item,category,reviews',
 '0,짜장면 ,중식 ,125',
 '1,짬뽕 ,중식 ,235',
 '2,김밥 ,분식 ,32',
 '3,떡볶이 ,분식 ,534',
 '4,라멘 ,일식 ,223',
 '5,돈가스 ,일식 ,52',
 '6,우동 ,일식 ,12 ']

In [12]:
header = lines.first()
filter_lines = lines.filter(lambda row: row != header)
filter_lines.collect()

['0,짜장면 ,중식 ,125',
 '1,짬뽕 ,중식 ,235',
 '2,김밥 ,분식 ,32',
 '3,떡볶이 ,분식 ,534',
 '4,라멘 ,일식 ,223',
 '5,돈가스 ,일식 ,52',
 '6,우동 ,일식 ,12 ']

In [14]:
def parse(filter_lines):
    row = filter_lines.split(',')
    return (str(row[2][:-1]), int(row[3]))

categoryReviews = filter_lines.map(parse)
categoryReviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12)]

In [15]:
categoryReviewsCount = categoryReviews.mapValues(lambda x : (x,1))
categoryReviewsCount.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1))]

In [16]:
reduced = categoryReviewsCount.reduceByKey(lambda x, y : (x[0]+y[0], x[1]+y[1]))
reduced.collect()

[('중식', (360, 2)), ('분식', (566, 2)), ('일식', (287, 3))]

In [17]:
averages = reduced.mapValues(lambda x : x[0]/x[1])
averages.collect()

[('중식', 180.0), ('분식', 283.0), ('일식', 95.66666666666667)]

24/04/02 16:04:25 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 582454 ms exceeds timeout 120000 ms
24/04/02 16:04:25 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/02 16:04:30 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at sc